In [ ]:
# =============================================================
# 06_train_image_model.ipynb — Multi-Architecture Full Training
# =============================================================
import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# -------------------------------------------------------------
# Environment setup
# -------------------------------------------------------------
print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus}")
else:
    print("⚠️ No GPU detected — training will use CPU (much slower).")

# -------------------------------------------------------------
# Parameters
# -------------------------------------------------------------
IMG_SIZE = (300, 300)
BATCH_SIZE = 16
EPOCHS = 3
SEED = 42

DATA_DIR = Path("data/processed/images/all_slices")
INDEX = Path("data/processed/images/all_index.csv")
SPLIT_DIR = Path("data/splits")
CLINICAL_PATH = Path("data/processed/clinical/clinical_features_with_id.csv")
SAVE_DIR = Path("results/images/multi_architecture")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------------------------------------------
# Load datasets + splits
# -------------------------------------------------------------
df = pd.read_csv(INDEX)
train_ids = pd.read_csv(SPLIT_DIR / "train_series.csv")["series_id"].tolist()
val_ids   = pd.read_csv(SPLIT_DIR / "val_series.csv")["series_id"].tolist()
test_ids  = pd.read_csv(SPLIT_DIR / "test_series.csv")["series_id"].tolist()

def assign_split(sid):
    if sid in train_ids: return "train"
    if sid in val_ids:   return "val"
    if sid in test_ids:  return "test"
    return "ignore"

df["split"] = df["series_id"].map(assign_split)
df = df[df["split"] != "ignore"]

print("🖼 Total image slices:", len(df))
print(df["split"].value_counts())

# -------------------------------------------------------------
# Merge with clinical metastasis labels
# -------------------------------------------------------------
clinical = pd.read_csv(CLINICAL_PATH)
clinical.columns = clinical.columns.str.strip()
meta_col = next((c for c in clinical.columns if "metastasis" in c.lower()), None)
if meta_col is None:
    raise SystemExit("❌ No metastasis column found in clinical data!")

merged = df.merge(clinical[["patient_id", meta_col]], on="patient_id", how="left")
possible_cols = [c for c in merged.columns if "metastasis" in c.lower()]
meta_col_final = possible_cols[0]
merged = merged.rename(columns={meta_col_final: "metastasis_status"})

merged["label"] = merged["metastasis_status"].replace({
    1: 1, 0: 0,
    "yes": 1, "metastatic": 1, "positive": 1,
    "no": 0, "non-metastatic": 0, "negative": 0
})
merged["label"] = pd.to_numeric(merged["label"], errors="coerce").fillna(0).astype(int)

print(f"✅ Labels assigned: {merged['label'].sum()} metastatic, {len(merged)-merged['label'].sum()} non-metastatic")
print("📊 Label distribution:\n", merged['label'].value_counts())

# -------------------------------------------------------------
# TensorFlow dataset builders
# -------------------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE

def decode_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return tf.cast(img, tf.float32) / 255.0

def make_dataset(subdf, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((subdf["slice_path"].values, subdf["label"].values))
    ds = ds.map(lambda p, y: (decode_img(p), y), num_parallel_calls=AUTOTUNE)
    if augment:
        ds = ds.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
        ds = ds.map(lambda x, y: (tf.image.random_brightness(x, 0.15), y))
    if shuffle:
        ds = ds.shuffle(2048, seed=SEED)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)


# -------------------------------------------------------------
# Subsample the dataset for a quick test run
# -------------------------------------------------------------
# Full-size:
# train: 19603, val: 4053, test: 4166
# Sample target: ~10% of each
train_df = merged[merged.split == "train"].sample(n=1960, random_state=42)
val_df   = merged[merged.split == "val"].sample(n=405, random_state=42)
test_df  = merged[merged.split == "test"].sample(n=416, random_state=42)

print(f"📊 Using subset for quick test:")
print(f"  Train: {len(train_df)}  Val: {len(val_df)}  Test: {len(test_df)}")

train_ds = make_dataset(train_df, augment=True)
val_ds   = make_dataset(val_df)
test_ds  = make_dataset(test_df, shuffle=False)

# train_ds = make_dataset(merged[merged.split=="train"], augment=True)
# val_ds   = make_dataset(merged[merged.split=="val"])
# test_ds  = make_dataset(merged[merged.split=="test"], shuffle=False)

print(f"✅ Datasets ready — Train: {len(merged[merged.split=='train'])}, Val: {len(merged[merged.split=='val'])}, Test: {len(merged[merged.split=='test'])}")

# -------------------------------------------------------------
# Safe logging helpers
# -------------------------------------------------------------
def safe_float(x):
    try:
        if hasattr(x, "numpy"):
            x = x.numpy()
        if isinstance(x, (np.ndarray, list, tuple)):
            return float(np.mean(x))
        return float(x)
    except Exception:
        return 0.0

class SafeJSONCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs:
            for k, v in logs.items():
                logs[k] = safe_float(v)

class SafeReduceLROnPlateau(tf.keras.callbacks.ReduceLROnPlateau):
    def on_epoch_end(self, epoch, logs=None):
        logs = {k: safe_float(v) for k, v in (logs or {}).items()}
        super().on_epoch_end(epoch, logs)

# =============================================================
# 🔄 Multi-Architecture Training
# =============================================================
architectures = {
    "EfficientNetB3": tf.keras.applications.EfficientNetB3,
    "EfficientNetV2S": tf.keras.applications.EfficientNetV2S,
    "DenseNet121": tf.keras.applications.DenseNet121,
    "ConvNeXtTiny": tf.keras.applications.ConvNeXtTiny
}

results_summary = []

for name, Backbone in architectures.items():
    print(f"\n🚀 Training model: {name}")
    tf.keras.backend.clear_session()

    base = Backbone(include_top=False, input_shape=IMG_SIZE + (3,), weights="imagenet", pooling="avg")
    base.trainable = True

    inputs = layers.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=True)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc"),
                 tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall")]
    )

    checkpoint_path = SAVE_DIR / f"{name}_best.weights.h5"

    callbacks = [
        SafeJSONCallback(),
        EarlyStopping(monitor="val_auc", patience=5, mode="max", restore_best_weights=True),
        ModelCheckpoint(filepath=str(checkpoint_path),
                        monitor="val_auc", save_best_only=True,
                        save_weights_only=True, mode="max", verbose=1),
        SafeReduceLROnPlateau(monitor="val_auc", factor=0.3, patience=3, verbose=1, mode="max")
    ]

    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks, verbose=1)
    results = model.evaluate(test_ds, return_dict=True)

    print(f"✅ Final Test Results ({name}):")
    for k, v in results.items():
        print(f"{k}: {v:.4f}")

    # Save training history and model
    pd.DataFrame(history.history).to_csv(SAVE_DIR / f"{name}_training_history.csv", index=False)
    tf.saved_model.save(model, str(SAVE_DIR / f"{name}_full_model"))
    results_summary.append({"model": name, **results})

# -------------------------------------------------------------
# Save summary table
# -------------------------------------------------------------
summary_df = pd.DataFrame(results_summary)
summary_path = SAVE_DIR / "architecture_comparison_summary.csv"
summary_df.to_csv(summary_path, index=False)
print("\n📊 Saved architecture comparison summary:")
print(summary_df)

# -------------------------------------------------------------
# Visualization
# -------------------------------------------------------------
plt.figure(figsize=(8, 5))
plt.bar(summary_df["model"], summary_df["accuracy"], label="Accuracy", alpha=0.7)
plt.bar(summary_df["model"], summary_df["auc"], label="AUC", alpha=0.7)
plt.title("Model Comparison: Accuracy & AUC")
plt.ylabel("Score")
plt.ylim(0.8, 1.0)
plt.legend()
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(SAVE_DIR / "architecture_comparison_barplot.png", dpi=300)
plt.show()

# =============================================================
# 🧠 Extract embeddings from best-performing model (by AUC)
# =============================================================
best_model_name = summary_df.loc[summary_df["auc"].idxmax(), "model"]
print(f"\n🏆 Best-performing model: {best_model_name}")

best_model_path = SAVE_DIR / f"{best_model_name}_full_model"
best_model = tf.keras.models.load_model(best_model_path)

feature_extractor = tf.keras.Model(inputs=best_model.input, outputs=best_model.layers[-2].output)

print(f"🔍 Extracting feature embeddings using {best_model_name}...")
all_paths = merged["slice_path"].tolist()
batch_size = 32
embeddings = []

for i in range(0, len(all_paths), batch_size):
    batch_imgs = [decode_img(p) for p in all_paths[i:i+batch_size]]
    batch_tensor = tf.stack(batch_imgs)
    emb = feature_extractor(batch_tensor).numpy()
    embeddings.append(emb)

embeddings = np.vstack(embeddings)
np.save(SAVE_DIR / f"{best_model_name}_embeddings.npy", embeddings)

pd.DataFrame({
    "slice_path": all_paths,
    "label": merged["label"].tolist()
}).to_csv(SAVE_DIR / f"{best_model_name}_embeddings_index.csv", index=False)

print(f"✅ Embeddings saved: {embeddings.shape} → {SAVE_DIR / f'{best_model_name}_embeddings.npy'}")
print("🎯 All training, evaluation, and embedding extraction complete.")


TensorFlow version: 2.12.0
⚠️ No GPU detected — training will use CPU (much slower).
🖼 Total image slices: 27822
split
train    19603
test      4166
val       4053
Name: count, dtype: int64
✅ Labels assigned: 1195 metastatic, 26627 non-metastatic
📊 Label distribution:
 label
0    26627
1     1195
Name: count, dtype: int64
📊 Using subset for quick test:
  Train: 1960  Val: 405  Test: 416
✅ Datasets ready — Train: 19603, Val: 4053, Test: 4166

🚀 Training model: EfficientNetB3
Epoch 1/3
  1/123 [..............................] - ETA: 2:35:30 - loss: 0.6898 - accuracy: 0.5625 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00

In [5]:
# =============================================================
# 06_train_image_model.ipynb — Full Model (Flexible Architecture)
# =============================================================
import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from pathlib import Path
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# -------------------------------------------------------------
# Environment setup
# -------------------------------------------------------------
print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus}")
else:
    print("⚠️ No GPU detected — training will use CPU (much slower).")

# -------------------------------------------------------------
# Parameters
# -------------------------------------------------------------
IMG_SIZE = (300, 300)
BATCH_SIZE = 16
EPOCHS = 3
SEED = 42

DATA_DIR = Path("data/processed/images/all_slices")
INDEX = Path("data/processed/images/all_index.csv")
SPLIT_DIR = Path("data/splits")
CLINICAL_PATH = Path("data/processed/clinical/clinical_features_with_id.csv")
SAVE_DIR = Path("results/images")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------------------------------------------
# Load datasets + splits
# -------------------------------------------------------------
df = pd.read_csv(INDEX)
train_ids = pd.read_csv(SPLIT_DIR / "train_series.csv")["series_id"].tolist()
val_ids   = pd.read_csv(SPLIT_DIR / "val_series.csv")["series_id"].tolist()
test_ids  = pd.read_csv(SPLIT_DIR / "test_series.csv")["series_id"].tolist()

def assign_split(sid):
    if sid in train_ids: return "train"
    if sid in val_ids:   return "val"
    if sid in test_ids:  return "test"
    return "ignore"

df["split"] = df["series_id"].map(assign_split)
df = df[df["split"] != "ignore"]

print("🖼 Total image slices:", len(df))
print(df["split"].value_counts())

# -------------------------------------------------------------
# Merge with clinical metastasis labels
# -------------------------------------------------------------
clinical = pd.read_csv(CLINICAL_PATH)
clinical.columns = clinical.columns.str.strip()
meta_col = next((c for c in clinical.columns if "metastasis" in c.lower()), None)
if meta_col is None:
    raise SystemExit("❌ No metastasis column found in clinical data!")

merged = df.merge(clinical[["patient_id", meta_col]], on="patient_id", how="left")

possible_cols = [c for c in merged.columns if "metastasis" in c.lower()]
meta_col_final = possible_cols[0]
merged = merged.rename(columns={meta_col_final: "metastasis_status"})

merged["label"] = merged["metastasis_status"].replace({
    1: 1, 0: 0,
    "yes": 1, "metastatic": 1, "positive": 1,
    "no": 0, "non-metastatic": 0, "negative": 0
})
merged["label"] = pd.to_numeric(merged["label"], errors="coerce").fillna(0).astype(int)

print(f"✅ Labels assigned: {merged['label'].sum()} metastatic, {len(merged)-merged['label'].sum()} non-metastatic")
print("📊 Label distribution:\n", merged['label'].value_counts())

# -------------------------------------------------------------
# TensorFlow dataset builders
# -------------------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE

def decode_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return tf.cast(img, tf.float32) / 255.0

def make_dataset(subdf, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((subdf["slice_path"].values, subdf["label"].values))
    ds = ds.map(lambda p, y: (decode_img(p), y), num_parallel_calls=AUTOTUNE)
    if augment:
        ds = ds.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
        ds = ds.map(lambda x, y: (tf.image.random_brightness(x, 0.15), y))
    if shuffle:
        ds = ds.shuffle(2048, seed=SEED)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# -------------------------------------------------------------
# Subsample the dataset for a quick test run
# -------------------------------------------------------------
# Full-size:
# train: 19603, val: 4053, test: 4166
# Sample target: ~10% of each
train_df = merged[merged.split == "train"].sample(n=1960, random_state=42)
val_df   = merged[merged.split == "val"].sample(n=405, random_state=42)
test_df  = merged[merged.split == "test"].sample(n=416, random_state=42)

print(f"📊 Using subset for quick test:")
print(f"  Train: {len(train_df)}  Val: {len(val_df)}  Test: {len(test_df)}")

train_ds = make_dataset(train_df, augment=True)
val_ds   = make_dataset(val_df)
test_ds  = make_dataset(test_df, shuffle=False)

print(f"✅ Datasets ready — Train: {len(merged[merged.split=='train'])}, Val: {len(merged[merged.split=='val'])}, Test: {len(merged[merged.split=='test'])}")

# -------------------------------------------------------------
# Clear session before building a big model
# -------------------------------------------------------------
tf.keras.backend.clear_session()

# -------------------------------------------------------------
# 🆕 Model definition (you can switch backbone here)
# -------------------------------------------------------------
# Available options: EfficientNetB3, EfficientNetV2S, DenseNet121, ConvNeXtTiny
Backbone = tf.keras.applications.EfficientNetB3   # <-- Change this line to test others

base = Backbone(
    include_top=False,
    input_shape=IMG_SIZE + (3,),
    weights="imagenet",
    pooling="avg"
)
base.trainable = True

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base(inputs, training=True)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc"),
             tf.keras.metrics.Precision(name="precision"),
             tf.keras.metrics.Recall(name="recall")]
)

model.summary()

# -------------------------------------------------------------
# Callbacks (JSON-safe + resume-ready)
# -------------------------------------------------------------
import json

def safe_float(x):
    try:
        if hasattr(x, "numpy"):
            x = x.numpy()
        if isinstance(x, (np.ndarray, list, tuple)):
            return float(np.mean(x))
        return float(x)
    except Exception:
        return 0.0

class SafeJSONCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs:
            for k, v in logs.items():
                logs[k] = safe_float(v)

class SafeReduceLROnPlateau(tf.keras.callbacks.ReduceLROnPlateau):
    def on_epoch_end(self, epoch, logs=None):
        logs = {k: safe_float(v) for k, v in (logs or {}).items()}
        super().on_epoch_end(epoch, logs)

checkpoint_path = SAVE_DIR / "best_image_model.weights.h5"

callbacks = [
    SafeJSONCallback(),
    EarlyStopping(monitor="val_auc", patience=5, mode="max", restore_best_weights=True),
    ModelCheckpoint(filepath=str(checkpoint_path),
                    monitor="val_auc", save_best_only=True,
                    save_weights_only=True, mode="max", verbose=1),
    SafeReduceLROnPlateau(monitor="val_auc", factor=0.3, patience=3, verbose=1, mode="max")
]

# -------------------------------------------------------------
# Resume from checkpoint if available
# -------------------------------------------------------------
if checkpoint_path.exists():
    print(f"🔁 Found existing checkpoint at {checkpoint_path}, loading weights...")
    model.load_weights(checkpoint_path)
else:
    print("🚀 Starting new training run from scratch...")

# -------------------------------------------------------------
# Determine last completed epoch
# -------------------------------------------------------------
history_path = SAVE_DIR / "training_history_best.csv"
initial_epoch = 0
if history_path.exists():
    hist = pd.read_csv(history_path)
    initial_epoch = len(hist)
    print(f"⏩ Resuming training from epoch {initial_epoch} of {EPOCHS}")

# -------------------------------------------------------------
# Train
# -------------------------------------------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    initial_epoch=initial_epoch,
    callbacks=callbacks,
    verbose=1
)

# -------------------------------------------------------------
# Evaluate
# -------------------------------------------------------------
results = model.evaluate(test_ds, return_dict=True)
print("\n✅ Final Test Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

pd.DataFrame(history.history).to_csv(SAVE_DIR / "training_history_best.csv", index=False)

# -------------------------------------------------------------
# Save model and feature embeddings
# -------------------------------------------------------------
print("\n💾 Saving model and feature embeddings...")

tf.saved_model.save(model, str(SAVE_DIR / "baseline_image_model_full_tf"))
print("✅ Model saved successfully (TensorFlow SavedModel format).")

# Extract feature embeddings
feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
all_paths = merged["slice_path"].tolist()
batch_size = 32
embeddings = []

for i in range(0, len(all_paths), batch_size):
    batch_imgs = [decode_img(p) for p in all_paths[i:i+batch_size]]
    batch_tensor = tf.stack(batch_imgs)
    emb = feature_extractor(batch_tensor).numpy()
    embeddings.append(emb)

embeddings = np.vstack(embeddings)
np.save(SAVE_DIR / "image_embeddings_full.npy", embeddings)
pd.DataFrame({
    "slice_path": all_paths,
    "label": merged["label"].tolist()
}).to_csv(SAVE_DIR / "image_embeddings_index_full.csv", index=False)

print(f"✅ Embeddings saved: {embeddings.shape} → {SAVE_DIR/'image_embeddings_full.npy'}")
print("🎯 Training complete.")



# =============================================================
# 🆕 NEW SECTION — Visualization of Results
# =============================================================

print("\n📊 Generating comparison plots...")

# --- Load summary file
summary = pd.read_csv(SAVE_DIR / "architecture_comparison_summary.csv")

# --- Accuracy & AUC bar plot
plt.figure(figsize=(8, 5))
plt.bar(summary["model"], summary["accuracy"], label="Accuracy", alpha=0.7)
plt.bar(summary["model"], summary["auc"], label="AUC", alpha=0.7)
plt.title("Model Comparison: Accuracy & AUC")
plt.ylabel("Score")
plt.ylim(0.8, 1.0)
plt.legend()
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(SAVE_DIR / "architecture_comparison_barplot.png", dpi=300)
plt.show()

# --- Training curves per model
for name in summary["model"]:
    hist_path = SAVE_DIR / f"{name}_training_history.csv"
    if hist_path.exists():
        hist = pd.read_csv(hist_path)
        plt.figure(figsize=(8, 4))
        plt.plot(hist["accuracy"], label="Train Accuracy")
        plt.plot(hist["val_accuracy"], label="Val Accuracy")
        plt.plot(hist["auc"], label="Train AUC")
        plt.plot(hist["val_auc"], label="Val AUC")
        plt.title(f"Training Curves — {name}")
        plt.xlabel("Epoch")
        plt.ylabel("Metric")
        plt.legend()
        plt.tight_layout()
        plt.savefig(SAVE_DIR / f"{name}_training_curves.png", dpi=300)
        plt.show()

print("\n✅ All plots saved to:", SAVE_DIR)

TensorFlow version: 2.12.0
⚠️ No GPU detected — training will use CPU (much slower).
🖼 Total image slices: 27822
split
train    19603
test      4166
val       4053
Name: count, dtype: int64
✅ Labels assigned: 1195 metastatic, 26627 non-metastatic
📊 Label distribution:
 label
0    26627
1     1195
Name: count, dtype: int64
📊 Using subset for quick test:
  Train: 1960  Val: 405  Test: 416
✅ Datasets ready — Train: 19603, Val: 4053, Test: 4166
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 dropout (Dropout)           (None, 1536)              0         
                                                                 
 dense (Dens

INFO:tensorflow:Assets written to: results\images\baseline_image_model_full_tf\assets


INFO:tensorflow:Assets written to: results\images\baseline_image_model_full_tf\assets


✅ Model saved successfully (TensorFlow SavedModel format).
✅ Embeddings saved: (27822, 1536) → results\images\image_embeddings_full.npy
🎯 Training complete.

📊 Generating comparison plots...


FileNotFoundError: [Errno 2] No such file or directory: 'results\\images\\architecture_comparison_summary.csv'

In [ ]:
# =============================================================
# 06_train_image_model.ipynb — Full EfficientNetB3 model
# =============================================================
import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# -------------------------------------------------------------
# Environment setup
# -------------------------------------------------------------
print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus}")
else:
    print("⚠️ No GPU detected — training will use CPU (much slower).")

# Parameters
IMG_SIZE = (300, 300)        # EfficientNetB3 default input
BATCH_SIZE = 16
EPOCHS = 3
SEED = 42

DATA_DIR = Path("data/processed/images/all_slices")
INDEX = Path("data/processed/images/all_index.csv")
SPLIT_DIR = Path("data/splits")
CLINICAL_PATH = Path("data/processed/clinical/clinical_features_with_id.csv")
SAVE_DIR = Path("results/images")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------------------------------------------
# Load datasets + splits
# -------------------------------------------------------------
df = pd.read_csv(INDEX)
train_ids = pd.read_csv(SPLIT_DIR / "train_series.csv")["series_id"].tolist()
val_ids   = pd.read_csv(SPLIT_DIR / "val_series.csv")["series_id"].tolist()
test_ids  = pd.read_csv(SPLIT_DIR / "test_series.csv")["series_id"].tolist()

def assign_split(sid):
    if sid in train_ids: return "train"
    if sid in val_ids:   return "val"
    if sid in test_ids:  return "test"
    return "ignore"

df["split"] = df["series_id"].map(assign_split)
df = df[df["split"] != "ignore"]

print("🖼 Total image slices:", len(df))
print(df["split"].value_counts())

# -------------------------------------------------------------
# Merge with clinical metastasis labels
# -------------------------------------------------------------
clinical = pd.read_csv(CLINICAL_PATH)
clinical.columns = clinical.columns.str.strip()
meta_col = next((c for c in clinical.columns if "metastasis" in c.lower()), None)
if meta_col is None:
    raise SystemExit("❌ No metastasis column found in clinical data!")

merged = df.merge(clinical[["patient_id", meta_col]], on="patient_id", how="left")

possible_cols = [c for c in merged.columns if "metastasis" in c.lower()]
meta_col_final = possible_cols[0]
merged = merged.rename(columns={meta_col_final: "metastasis_status"})

merged["label"] = merged["metastasis_status"].replace({
    1: 1, 0: 0,
    "yes": 1, "metastatic": 1, "positive": 1,
    "no": 0, "non-metastatic": 0, "negative": 0
})
merged["label"] = pd.to_numeric(merged["label"], errors="coerce").fillna(0).astype(int)

print(f"✅ Labels assigned: {merged['label'].sum()} metastatic, {len(merged)-merged['label'].sum()} non-metastatic")
print("📊 Label distribution:\n", merged['label'].value_counts())

# -------------------------------------------------------------
# TensorFlow dataset builders
# -------------------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE

def decode_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return tf.cast(img, tf.float32) / 255.0

def make_dataset(subdf, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((subdf["slice_path"].values, subdf["label"].values))
    ds = ds.map(lambda p, y: (decode_img(p), y), num_parallel_calls=AUTOTUNE)
    if augment:
        ds = ds.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
        ds = ds.map(lambda x, y: (tf.image.random_brightness(x, 0.15), y))
    if shuffle:
        ds = ds.shuffle(2048, seed=SEED)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# -------------------------------------------------------------
# Subsample the dataset for a quick test run
# -------------------------------------------------------------
# Full-size:
# train: 19603, val: 4053, test: 4166
# Sample target: ~10% of each
train_df = merged[merged.split == "train"].sample(n=3920, random_state=42)
val_df   = merged[merged.split == "val"].sample(n=810, random_state=42)
test_df  = merged[merged.split == "test"].sample(n=833, random_state=42)

print(f"📊 Using subset for quick test:")
print(f"  Train: {len(train_df)}  Val: {len(val_df)}  Test: {len(test_df)}")

train_ds = make_dataset(train_df, augment=True)
val_ds   = make_dataset(val_df)
test_ds  = make_dataset(test_df, shuffle=False)

print(f"✅ Datasets ready — Train: {len(merged[merged.split=='train'])}, Val: {len(merged[merged.split=='val'])}, Test: {len(merged[merged.split=='test'])}")

# -------------------------------------------------------------
# Model definition — EfficientNetB3
# -------------------------------------------------------------
base = tf.keras.applications.EfficientNetB3(
    include_top=False,
    input_shape=IMG_SIZE + (3,),
    weights="imagenet",
    pooling="avg"
)
base.trainable = True   # fine-tune all layers (can freeze first N later)

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base(inputs, training=True)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)

model.summary()

# -------------------------------------------------------------
# Callbacks (With JSON-safe and weight-only checkpoints)
# -------------------------------------------------------------
import json
import numpy as np

def safe_float(x):
    """Convert any metric value (tensor/ndarray) to plain float."""
    try:
        if hasattr(x, "numpy"):
            x = x.numpy()
        if isinstance(x, (np.ndarray, list, tuple)):
            return float(np.mean(x))
        return float(x)
    except Exception:
        return 0.0

class SafeJSONCallback(tf.keras.callbacks.Callback):
    """Prevent JSON serialization errors in Keras history."""
    def on_epoch_end(self, epoch, logs=None):
        if logs:
            for k, v in logs.items():
                logs[k] = safe_float(v)

class SafeReduceLROnPlateau(tf.keras.callbacks.ReduceLROnPlateau):
    def on_epoch_end(self, epoch, logs=None):
        logs = {k: safe_float(v) for k, v in (logs or {}).items()}
        super().on_epoch_end(epoch, logs)

# Weight-only checkpoint file (robust across TF versions)
checkpoint_path = SAVE_DIR / "best_image_model.weights.h5"

callbacks = [
    SafeJSONCallback(),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_auc", patience=5, mode="max", restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_path),
        monitor="val_auc", save_best_only=True,
        save_weights_only=True,   # ✅ prevents broken .keras files
        mode="max", verbose=1),
    SafeReduceLROnPlateau(
        monitor="val_auc", factor=0.3, patience=3, verbose=1, mode="max"),
]

# Optional safety: clear old graphs before starting/continuing a long run
tf.keras.backend.clear_session()

# -------------------------------------------------------------
# Resume from checkpoint if available
# -------------------------------------------------------------
if checkpoint_path.exists():
    print(f"🔁 Found existing checkpoint at {checkpoint_path}, resuming training (loading weights only)...")
    model.load_weights(checkpoint_path)
else:
    print("🚀 Starting new training run from scratch...")

# -------------------------------------------------------------
# Determine last completed epoch
# -------------------------------------------------------------
history_path = SAVE_DIR / "training_history_best.csv"
initial_epoch = 0
if history_path.exists():
    hist = pd.read_csv(history_path)
    initial_epoch = len(hist)
    print(f"⏩ Resuming training from epoch {initial_epoch} of {EPOCHS}")

# -------------------------------------------------------------
# Train
# -------------------------------------------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    initial_epoch=initial_epoch,
    callbacks=callbacks,
    verbose=1
)
# -------------------------------------------------------------
# Evaluate
# -------------------------------------------------------------
results = model.evaluate(test_ds, return_dict=True)
print("\n✅ Final Test Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

pd.DataFrame(history.history).to_csv(SAVE_DIR / "training_history_best.csv", index=False)

# -------------------------------------------------------------
# Save model and embeddings
# -------------------------------------------------------------
print("\n💾 Saving model and feature embeddings...")

# Save in pure TensorFlow SavedModel format (robust & safe)
tf.saved_model.save(model, str(SAVE_DIR / "baseline_image_model_full_tf"))
print("✅ Model saved successfully (TensorFlow SavedModel format).")


#  I WANT TO IGNORE THIS PART FOR NOW, because it works on the all dataset and have a large runtime.

# # Extract feature embeddings
# feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
# all_paths = merged["slice_path"].tolist()
# batch_size = 32
# embeddings = []

# for i in range(0, len(all_paths), batch_size):
#     batch_imgs = [decode_img(p) for p in all_paths[i:i+batch_size]]
#     batch_tensor = tf.stack(batch_imgs)
#     emb = feature_extractor(batch_tensor).numpy()
#     embeddings.append(emb)

# embeddings = np.vstack(embeddings)
# np.save(SAVE_DIR / "image_embeddings_full.npy", embeddings)
# pd.DataFrame({
#     "slice_path": all_paths,
#     "label": merged["label"].tolist()
# }).to_csv(SAVE_DIR / "image_embeddings_index_full.csv", index=False)

# print(f"✅ Embeddings saved: {embeddings.shape} → {SAVE_DIR/'image_embeddings_full.npy'}")
# print("🎯 Training complete.")


c:\Users\Negar\anaconda3\envs\crlm\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


TensorFlow version: 2.12.0
⚠️ No GPU detected — training will use CPU (much slower).
🖼 Total image slices: 27822
split
train    19603
test      4166
val       4053
Name: count, dtype: int64
✅ Labels assigned: 1195 metastatic, 26627 non-metastatic
📊 Label distribution:
 label
0    26627
1     1195
Name: count, dtype: int64
📊 Using subset for quick test:
  Train: 3920  Val: 810  Test: 833
✅ Datasets ready — Train: 19603, Val: 4053, Test: 4166
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 dropout (Dropout)           (None, 1536)              0         
                                                                 
 dense (Dens